# Test - Data Availability (RS)
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* May 13, 2022

In [4]:
import boto3 
import pandas as pd
import psycopg2

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

### Create a connection to RedShift PROD:

In [3]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

MGKP-XHQR
Successully logged into Start URL: https://stellaralgo.awsapps.com/start


In [8]:
session = boto3.session.Session(profile_name='Stellaralgo-DataScienceAdmin')
client = session.client('redshift')

CLUSTER = 'qa-app'
DBNAME = 'stlrmls'

creds = client.get_cluster_credentials(                
    ClusterIdentifier = CLUSTER,
    DbUser = 'admin',
    DbName = DBNAME,
    DbGroups = ['admin_group'],
    AutoCreate=True
)

print(f"GOT CONNECTION TO DATABASE: {CLUSTER} {DBNAME}")

GOT CONNECTION TO DATABASE: qa-app stlrmls


### Run the stored proc to get retention training and inference datasets: (! NEEDS ROLE ASSUMPTION !)

In [16]:
# create a connection & cursor and call the stored procedure
conn = psycopg2.connect(
    host = 'qa-app.ctjussvyafp4.us-east-1.redshift.amazonaws.com',
    port = 5439,
    user = creds['DbUser'],
    password = creds['DbPassword'],
    database = DBNAME
)

cur = conn.cursor()
cur.execute(f"CALL dw.getretentionmodeldata(31, 2020, 2022, 'rkcursor')")

# create a named cursor based on the cursor name passed in above
named_cursor = conn.cursor('rkcursor')
data = named_cursor.fetchall()

# load the data and columns into a data frame
cols = [row[0] for row in named_cursor.description]
df = pd.DataFrame(data=data, columns=cols)

conn.commit()

### Check how much data we have for each season:

In [19]:
print(f"DATA AVAILABILITY REPORT:")
print(f" > {CLUSTER} {DBNAME}")
df["year"].value_counts().sort_index(ascending=False)

DATA AVAILABILITY REPORT:
 > qa-app stlrmls


2022    1741
2021    2073
2020    1900
Name: year, dtype: int64

### Done